In [9]:
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd
import numpy as np

In [10]:
url = "http://www.com.tw/cross/university_017_102.html"

res = requests.get(url)
soup = bs(res.text,"html5lib")
tb = soup.select('#table1')[0]

In [11]:
dfs = pd.read_html(tb.prettify('utf-8'), encoding='utf-8')[0]



dfs = dfs.dropna(how="all",axis=(0,1)).iloc[1:,:2]


dfs.columns = dfs.iloc[0,:].values.tolist()

dfs = dfs.iloc[1:,:].reset_index(drop=True)

dfs.iloc[:,0] = dfs.iloc[:,0].str[1:-1]

In [5]:
# dfs.to_csv("代碼.csv",index=False)

In [98]:
dfs.iloc[:,0].values[15:16]

array(['017162'], dtype=object)

In [140]:
#url2 = "http://www.com.tw/cross/check_{}_NO_1_{}_0_3.html"  可以再寫個format
url2 = "http://www.com.tw/cross/check_{}_NO_1_102_0_3.html"
# 網址
import time
# for dnum in dfs.iloc[:,0].values[15:16]: 測試用這行
for dnum in dfs.iloc[:,0].values:
    
    time.sleep(2) # 睡個兩秒免得被抄
    res2 = requests.get(url2.format(dnum))

    soup2 = bs(res2.text,"html5lib")
    # 選擇第一張table
    tb2 = soup2.select('#mainContent table')[0]
    # 從table 選擇有bgcolor的屬性的tr(90員)
    realtr = tb2.select("tr[bgcolor]")
    # 從table 選擇tr裡的table(90員)
    table2 = tb2.select("table")
    totaldf = pd.DataFrame() # 空df用來接全部
    for u in np.arange(len(table2)):
        num=realtr[u].select("td div")[0].text
        sID=realtr[u].select("td div")[1].text.split("考區 :")[0].strip()
        school=realtr[u].select("td div")[1].text.split("考區 :")[1].strip()
        name=realtr[u].select("td")[3].text.strip()
        tesdfs = pd.read_html(table2[u].prettify('utf-8'), encoding='utf-8')[0]
        for i in table2[u].select("tr"):
            if i.findChildren('img'):
                tesdfs.loc[tesdfs.loc[:,1] == i.select("a")[0].text,0] = "入取"
                tesdfs["num"] = num
                tesdfs["sID"] = sID
                tesdfs["school"] = school
                tesdfs["name"] = name
                totaldf = totaldf.append(tesdfs,ignore_index=True)

    totaldf.fillna("0",inplace=True)

    totaldf.columns=["in_or_not","departname","in_num","this_num","sID","hschool","name"]
    totaldf.to_csv(dfs.loc[dfs["校系代碼"]==dnum,"系名"].values[0]+"csv",index=False)

In [144]:
totaldf.loc[~totaldf.duplicated()]

,in_or_not,departname,in_num,this_num,sID,hschool,name
0,入取,中國文化大學地質學系,正1,正1,10166336,新竹市立建功高中,蘇泓予
1,0,中國文化大學地質學系,正2,正2,10088214,新北市立板橋高中,方銘陽
2,0,中國文化大學森林暨自然保育學系,正21,正2,10088214,新北市立板橋高中,方銘陽
3,0,義守大學土木與生態工程學系,正取,正2,10088214,新北市立板橋高中,方銘陽
4,入取,義守大學材料科學與工程學系,正取,正2,10088214,新北市立板橋高中,方銘陽
5,0,中國文化大學地質學系,正3,正3,10243141,國立雲林科技大學,林韋良
6,入取,中國文化大學地理學系,正9,正3,10243141,國立雲林科技大學,林韋良
7,0,中國文化大學園藝暨生物技術學系,正14,正3,10243141,國立雲林科技大學,林韋良
8,0,中國文化大學森林暨自然保育學系,正25,正3,10243141,國立雲林科技大學,林韋良
9,0,國立金門大學護理學系,備22,正3,10243141,國立雲林科技大學,林韋良


In [74]:
realtr = tb2.select("tr[bgcolor]")
table2 = tb2.select("table")
totaldf = pd.DataFrame()
for u in np.arange(len(table2)):
    num=realtr[u].select("td div")[0].text
    sID=realtr[u].select("td div")[1].text.split("考區 :")[0].strip()
    school=realtr[u].select("td div")[1].text.split("考區 :")[1].strip()
    name=realtr[u].select("td")[3].text.strip()
    tesdfs = pd.read_html(table2[u].prettify('utf-8'), encoding='utf-8')[0]
    for i in table2[u].select("tr"):
        if i.findChildren('img'):
            tesdfs.loc[tesdfs.loc[:,1] == i.select("a")[0].text,0] = "入取"
            tesdfs["num"] = num
            tesdfs["sID"] = sID
            tesdfs["school"] = school
            tesdfs["name"] = name
            totaldf = totaldf.append(tesdfs,ignore_index=True)

totaldf.fillna("0",inplace=True)

totaldf.columns=["in_or_not","departname","in_num","this_num","sID","hschool","name"]